# 1.0 Data Cleaning Script

## 1.1 Importing Stuff

Import date of raw data file `CA.US-Social.csv`:  2024-11-17

Source:  https://webapps.cihr-irsc.gc.ca/decisions/p/main.html?lang=en#fq={!tag=theme2}theme2%3A%22Social%20%2F%20Cultural%20%2F%20Environmental%20%2F%20Population%20Health%22&fq={!tag=country}country%3ACanada%20%20%20OR%20%20%20country%3A%22United%20States%20of%20America%22&sort=namesort%20asc&start=0&rows=20

In [11]:
# installations
## section 1.1
import pandas as pd

## section 1.2
from bs4 import BeautifulSoup
import csv
import numpy as np
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

## section 1.3
import os

In [33]:
data = pd.read_csv("raw data/CA.US-Social.csv", encoding='latin1')
grant_count = data.shape[0] # will use this in section 1.2

In [27]:
# clean numeric columns
numeric_cols = ['CIHR_Contribution', 'CIHR_Equipment']
for column in numeric_cols:
    data[column] = data[column].replace({'\$': '', ',': ''}, regex=True)

# remove 0's
data['CIHR_Contribution'] = pd.to_numeric(data['CIHR_Contribution'], errors='coerce') 
data = data.query("CIHR_Contribution != 0")

## 1.2 Scraping Data

So the search results from which I imported the data contains links to the information on the papers.  Within these links, the abstracts can be found, so once we can scrape that we're golden.

Issues:
- search results are split into 352 pages, with 20 results per page
- the links to the papers are independent of their positions on the search results

Link structure of search results:

`https://webapps.cihr-irsc.gc.ca/decisions/p/main.html?lang=en#fq={!tag=theme2}theme2%3A%22Social%20%2F%20Cultural%20%2F%20Environmental%20%2F%20Population%20Health%22&fq={!tag=country}country%3ACanada%20%20%20OR%20%20%20country%3A%22United%20States%20of%20America%22&sort=namesort%20asc&start=`**0**`&rows=20` where `row` is a multiple of 20.  The maximum n should be the  max possible integer that satisfies 20n < [count of grants] 



Link structure of papers:

`https://webapps.cihr-irsc.gc.ca/decisions/p/project_details.html?applId=`**462354**`&lang=en` where the `applID` is unique to each paper.

Steps:
1. download chromedriver here https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.69/win64/chromedriver-win64.zip and extract.  The extract location will differ based on operating system (Windows or Mac)
2. initialize driver for selenium.  This step is required since the site is dynamically loaded
3. load and scrape the `applID`s of all n pages of the search result
    
    a. save to `data/applids.csv`
4. use the `applID` to obtain the soup for each grant
5. round 1 of obtaining the abstract, project name, researcher(s) names, contribution count from the soup
6. round 2 of obtaining stuff, since some of the webscraping failed
7. round 3 of obtaining stuff aghhhhh
8. merge everything from grants_scraped1, 2, 3... into one df

### 1.2.1 Scraping applIDs

This shit took 27 mins to run.  Don't run this section again unless necessary LUL

In [ ]:
# Calculate number of pages
if grant_count % 20 == 0:
    page_count = grant_count // 20  # No extra page needed if divisible by 20
else:
    page_count = grant_count // 20 + 1  # Add one more page if not divisible by 20

print(f"Total pages to scrape: {page_count}")

In [ ]:
# Set up Selenium WebDriver with the correct path
options = Options()
options.add_argument('--headless')
service = Service("C:/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
applids = []

for page_num in range(page_count):
    try:
        start = str(page_num * 20)
        link = 'https://webapps.cihr-irsc.gc.ca/decisions/p/main.html?lang=en#fq={!tag=theme2}theme2%3A%22Social%20%2F%20Cultural%20%2F%20Environmental%20%2F%20Population%20Health%22&fq={!tag=country}country%3ACanada%20%20%20OR%20%20%20country%3A%22United%20States%20of%20America%22&sort=namesort%20asc&start=' + start + '&rows=20'
        driver.get(link)
        driver.implicitly_wait(15)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', class_='pull-left')
        for link in links:
            href = link.get('href', '')
            match = re.search(r'applId=(\d+)', href)
            # lang = re.search(r'lang=(\d+)', href)
            if match:
                applids.append(match.group(1))
        # Add a delay between requests to avoid overloading the server and getting my ass banned
        delay = random.uniform(3, 6)
        time.sleep(delay)
    except Exception as e:
        print(f"Error occurred on page {page_num}. Error details: {str(e)}")
        break

In [ ]:
driver.quit()

with open('data/applids.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['applId'])  # Write header
    for applid in applids:
        writer.writerow([applid])

### 1.2.2 Scraping everything else

In [2]:
# initialize and prepare stuff
grants_info = pd.read_csv("data/applids.csv")
grants_applIDs = grants_info['applId'].to_list()

# Set up Selenium WebDriver with the correct path
options = Options()
options.add_argument('--headless')
service = Service("C:/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# define functions
def extract_info(dl):
    data = {}
    items = dl.find_all(["dt", "dd"])
    for i in range(0, len(items), 2):  # Iterate in pairs of dt and dd
        label = items[i].get_text(strip=True).replace(":", "")
        value = items[i + 1].get_text(strip=True)
        data[label] = value
    return data

def clean_contribution(data):
    contribution_str = data['CIHR contribution']
    if isinstance(contribution_str, str):
        match = re.search(r'Amount:\$(\d{1,3}(?:,\d{3})*)', contribution_str)
        if match:
            data['CIHR contribution'] = float(match.group(1).replace(',', ''))
        else:
            data['CIHR contribution'] = None
    else:
        pass

def convert_to_list_of_strings(data, key):
    # Check if the value for the given key is a string
    if isinstance(data.get(key), str):
        # Split the string by semicolon and remove extra spaces
        data[key] = [name.strip() for name in data[key].split(';')]

In [4]:
# define other shit
grants_scraped = pd.DataFrame(columns=["Project title", "Principal investigator(s)", "Co-investigator(s)", "Keywords", "CIHR contribution", "ApplID"])

keys_to_remove = ['Supervisors', 'Institution paid', 'Research institution', 'Department', 'External applicant partner(s)', 'Program', 'Competition (year/month)', 
                  'Assigned peer review committee', 'Primary institute', 'Primary theme', 'Term (yrs/mths)', 'Contributors', 'Amount', 'Equipment', 'External funding partner(s)', 
                  'Partner Name', 'External applicant partner(s)', 'External in-kind partner(s)']

chunks = np.array_split(grants_applIDs, len(grants_applIDs) // 662 + 1)
chunk_size = 662
chunks = [grants_applIDs[i:i + chunk_size] for i in range(0, len(grants_applIDs), chunk_size)]

# Now, `chunks` is a list of lists, where each sublist contains 662 entries (or fewer for the last one)

#### 1.2.2.1 Round 1

In [9]:
grants_scraped = pd.DataFrame(columns=["Project title", "Principal investigator(s)", "Co-investigator(s)", "Keywords", "CIHR contribution", "ApplID"])  # Initialize DataFrame
failed = []

# round 1
for id in chunks[9]:
    try:
        link = 'https://webapps.cihr-irsc.gc.ca/decisions/p/project_details.html?applId=' + str(id) + '&lang=en'
        driver.get(link)
        driver.implicitly_wait(30)
        html = driver.page_source
        if not html:
            failed.append(id)  # Add to the failed list
            continue  # Skip this iteration if the content is missing
        soup = BeautifulSoup(html, 'html.parser')
        dl = soup.find("dl", id="results")
        if not dl:
            failed.append(id)  # Add to the failed list
            continue
        data = extract_info(dl)
        if not data:  # If the dictionary is empty, skip this iteration
            failed.append(id)  # Add to the failed list
            continue
        for key in keys_to_remove:
            data.pop(key, None)

        # Clean and convert as needed
        if 'CIHR contribution' in data:
            clean_contribution(data)
        if 'Principal investigator(s)' in data:
            convert_to_list_of_strings(data, 'Principal investigator(s)')
        if 'Co-investigator(s)' in data:
            convert_to_list_of_strings(data, 'Co-investigator(s)')
        if 'Keywords' in data:
            convert_to_list_of_strings(data, 'Keywords')

        data['ApplID'] = id  # Add ApplID to the data dictionary

        # Convert dictionary to DataFrame and concatenate
        grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)

        # Add delay between requests
        delay = random.uniform(3, 6)
        time.sleep(delay)

    except Exception as e:
        print(f"Error details: {str(e)}")
        break  # Break if an exception occurs

C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\496615978.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)
C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\496615978.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)
C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\496615978.py:39: FutureWarning: The behavior of Data

In [10]:
grants_scraped.to_csv('data/grants_scraped10.csv', index=False)
failed_df = pd.DataFrame(failed, columns=["Failed_Ids"])
failed_df.to_csv('data/failed_ids_chunk10.csv', index=False)

In [11]:
driver.quit()

#### 1.2.2.2 Round 2

In [14]:
# load failed IDs
failed_ids1 = pd.read_csv("data/failed_ids_chunk1.csv")
failed_ids2 = pd.read_csv("data/failed_ids_chunk2.csv")
failed_ids3 = pd.read_csv("data/failed_ids_chunk3.csv")
failed_ids4 = pd.read_csv("data/failed_ids_chunk4.csv")
failed_ids5 = pd.read_csv("data/failed_ids_chunk5.csv")
failed_ids6 = pd.read_csv("data/failed_ids_chunk6.csv")
failed_ids7 = pd.read_csv("data/failed_ids_chunk7.csv")
failed_ids8 = pd.read_csv("data/failed_ids_chunk8.csv")
failed_ids9 = pd.read_csv("data/failed_ids_chunk9.csv")
failed_ids10 = pd.read_csv("data/failed_ids_chunk10.csv")

combined_failed_ids = pd.concat([failed_ids1["Failed_Ids"], failed_ids2["Failed_Ids"], failed_ids3["Failed_Ids"], failed_ids4["Failed_Ids"], 
                                 failed_ids5["Failed_Ids"], failed_ids6["Failed_Ids"], failed_ids7["Failed_Ids"], failed_ids8["Failed_Ids"], 
                                 failed_ids9["Failed_Ids"], failed_ids10["Failed_Ids"]]).tolist()
combined_failed_ids

[453092,
 476159,
 460914,
 391549,
 468873,
 314478,
 453430,
 403880,
 462354,
 228923,
 228924,
 255123,
 404498,
 430689,
 376241,
 268801,
 494087,
 410620,
 257386,
 209010,
 403425,
 260382,
 264389,
 478921,
 451189,
 459720,
 328403,
 313232,
 254868,
 339780,
 334751,
 488644,
 262671,
 366158,
 408284,
 443820,
 489564,
 458885,
 476178,
 314887,
 194028,
 450562,
 185447,
 451629,
 441569,
 338166,
 175965,
 278524,
 321724,
 308847,
 191821,
 363358,
 460182,
 336942,
 360780,
 297486,
 383129,
 457458,
 283883,
 499259,
 369985,
 450580,
 475914,
 476117,
 193465,
 240707,
 215932,
 254826,
 226072,
 313617,
 397066,
 366547,
 477582,
 231331,
 348265,
 362592,
 331866,
 459480,
 279991,
 331881,
 363916,
 362038,
 480902,
 487912,
 412147,
 399652,
 504809,
 450318,
 297492,
 347121,
 243815,
 281154,
 328706,
 485623,
 489155,
 380307,
 422851,
 245970,
 459244,
 202002,
 229682,
 404097,
 302030,
 280120,
 504136,
 411878,
 447327,
 352439,
 365531,
 182203,
 361981,
 

In [ ]:
grants_scraped = pd.DataFrame(columns=["Project title", "Principal investigator(s)", "Co-investigator(s)", "Keywords", "CIHR contribution", "ApplID"])  # Initialize DataFrame
failed = []

for id in combined_failed_ids:
    try:
        link = 'https://webapps.cihr-irsc.gc.ca/decisions/p/project_details.html?applId=' + str(id) + '&lang=en'
        driver.get(link)
        driver.implicitly_wait(30)
        html = driver.page_source
        if not html:
            failed.append(id)  # Add to the failed list
            continue  # Skip this iteration if the content is missing
        soup = BeautifulSoup(html, 'html.parser')
        dl = soup.find("dl", id="results")
        if not dl:
            failed.append(id)  # Add to the failed list
            continue
        data = extract_info(dl)
        if not data:  # If the dictionary is empty, skip this iteration
            failed.append(id)  # Add to the failed list
            continue
        for key in keys_to_remove:
            data.pop(key, None)

        # Clean and convert as needed
        if 'CIHR contribution' in data:
            clean_contribution(data)
        if 'Principal investigator(s)' in data:
            convert_to_list_of_strings(data, 'Principal investigator(s)')
        if 'Co-investigator(s)' in data:
            convert_to_list_of_strings(data, 'Co-investigator(s)')
        if 'Keywords' in data:
            convert_to_list_of_strings(data, 'Keywords')

        data['ApplID'] = id  # Add ApplID to the data dictionary

        # Convert dictionary to DataFrame and concatenate
        grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)

        # Add delay between requests
        delay = random.uniform(3, 6)
        time.sleep(delay)

    except Exception as e:
        print(f"Error details: {str(e)}")
        break  # Break if an exception occurs

C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\723926142.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)
C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\723926142.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)
C:\Users\fuzzi\AppData\Local\Temp\ipykernel_18108\723926142.py:39: FutureWarning: The behavior of Data

In [ ]:
grants_scraped.to_csv('data/grants_scraped_from_round2.csv', index=False)
failed_df = pd.DataFrame(failed, columns=["Failed_Ids"])
failed_df.to_csv('data/failed_ids_round2.csv', index=False)

#### 1.2.2.3 Round 3

In [5]:
# load failed IDs
failed_ids_from_round2 = pd.read_csv("data/failed_ids_round2.csv")

combined_failed_ids = pd.concat([failed_ids_from_round2["Failed_Ids"]]).tolist()
combined_failed_ids

[268801,
 328706,
 229648,
 465979,
 334192,
 277269,
 475158,
 404078,
 207757,
 209428,
 238440,
 263505,
 313958,
 378425]

In [9]:
# grants_scraped = pd.DataFrame(columns=["Project title", "Principal investigator(s)", "Co-investigator(s)", "Keywords", "CIHR contribution", "ApplID"])  # Initialize DataFrame
failed2 = []

for id in failed1:
    try:
        link = 'https://webapps.cihr-irsc.gc.ca/decisions/p/project_details.html?applId=' + str(id) + '&lang=en'
        driver.get(link)
        driver.implicitly_wait(30)
        html = driver.page_source
        if not html:
            failed2.append(id)  # Add to the failed list
            continue  # Skip this iteration if the content is missing
        soup = BeautifulSoup(html, 'html.parser')
        dl = soup.find("dl", id="results")
        if not dl:
            failed2.append(id)  # Add to the failed list
            continue
        data = extract_info(dl)
        if not data:  # If the dictionary is empty, skip this iteration
            failed2.append(id)  # Add to the failed list
            continue
        for key in keys_to_remove:
            data.pop(key, None)

        # Clean and convert as needed
        if 'CIHR contribution' in data:
            clean_contribution(data)
        if 'Principal investigator(s)' in data:
            convert_to_list_of_strings(data, 'Principal investigator(s)')
        if 'Co-investigator(s)' in data:
            convert_to_list_of_strings(data, 'Co-investigator(s)')
        if 'Keywords' in data:
            convert_to_list_of_strings(data, 'Keywords')

        data['ApplID'] = id  # Add ApplID to the data dictionary

        # Convert dictionary to DataFrame and concatenate
        grants_scraped = pd.concat([grants_scraped, pd.DataFrame([data])], ignore_index=True)

        # Add delay between requests
        delay = random.uniform(3, 6)
        time.sleep(delay)

    except Exception as e:
        print(f"Error details: {str(e)}")
        break  # Break if an exception occurs

In [10]:
grants_scraped.to_csv('data/grants_scraped_from_round3.csv', index=False)
# failed_df = pd.DataFrame(failed, columns=["Failed_Ids"])
# failed_df.to_csv('data/failed_ids_round3.csv', index=False)

## 1.3 Merge Results into one DF

In [14]:
# Specify the directory for the output
input_dir = "data/data chunks"  # Directory containing the CSV files
output_dir = "data"  # Directory where the merged file will be saved
output_file = os.path.join(output_dir, "grants.csv")

os.makedirs(output_dir, exist_ok=True)

In [15]:
# Get a list of all CSV files in the directory
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
csv_files

['grants_scraped1.csv',
 'grants_scraped10.csv',
 'grants_scraped2.csv',
 'grants_scraped3.csv',
 'grants_scraped4.csv',
 'grants_scraped5.csv',
 'grants_scraped6.csv',
 'grants_scraped7.csv',
 'grants_scraped8.csv',
 'grants_scraped9.csv',
 'grants_scraped_from_round2.csv',
 'grants_scraped_from_round3.csv']

In [16]:
# do merge
merged_df = pd.DataFrame(columns=["Project title", "Principal investigator(s)", 
                                  "Co-investigator(s)", "Keywords", 
                                  "CIHR contribution", "ApplID"])

for file in csv_files:
    file_path = os.path.join(input_dir, file)
    df = pd.read_csv(file_path)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

merged_df.to_csv(output_file, index=False)

print(f"Merged CSV saved to {output_file}")

C:\Users\fuzzi\AppData\Local\Temp\ipykernel_17668\3694851484.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([merged_df, df], ignore_index=True)


Merged CSV saved to data\grants.csv
